In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

1. 얼굴 질병 예측 모델

In [ ]:
from pathlib import Path
train_dir = Path(r'D:\main\train')
test_dir = Path(r'D:\main\testing')


In [ ]:
Acne = list(train_dir.glob('Acne/*'))
Actinic = list(train_dir.glob('Actinic Keratosis/*'))
Basal = list(train_dir.glob('Basal Cell Carcinoma/*'))
Eczemaa = list(train_dir.glob('Eczemaa/*'))
Rosacea = list(train_dir.glob('Rosacea/*'))


In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = len(class_names)

facemodel = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
facemodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=15
history = facemodel.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

2. 여드름 심각도 측정 모델

In [77]:
from pathlib import Path
data_dir = Path(r'D:\Created Dataset')

In [78]:
Level_0 = list(data_dir.glob('Level_0/*'))
Level_1 = list(data_dir.glob('Level_1/*'))
Level_2 = list(data_dir.glob('Level_2/*'))
Level_3 = list(data_dir.glob('Level_3/*'))


In [79]:
batch_size = 32
img_height = 180
img_width = 180

In [88]:
train2_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val2_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1006 files belonging to 4 classes.
Using 805 files for training.
Found 1006 files belonging to 4 classes.
Using 201 files for validation.


In [89]:
normalization2_layer = layers.Rescaling(1./255)
normalized2_ds = train2_ds.map(lambda x, y: (normalization2_layer(x), y))
image_batch, labels_batch = next(iter(normalized2_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.9416667


In [90]:
num2_classes = len(train2_ds.class_names)

levelmodel = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num2_classes)
])

In [75]:
levelmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [76]:
epochs=10
history = levelmodel.fit(
  train2_ds,
  validation_data=val2_ds,
  epochs=epochs
)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 466ms/step - accuracy: 0.4444 - loss: 1.1381 - val_accuracy: 0.4677 - val_loss: 1.0632
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 401ms/step - accuracy: 0.4830 - loss: 1.0427 - val_accuracy: 0.4677 - val_loss: 1.0251
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 397ms/step - accuracy: 0.4428 - loss: 0.9948 - val_accuracy: 0.4378 - val_loss: 1.0117
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 420ms/step - accuracy: 0.5465 - loss: 0.9069 - val_accuracy: 0.4776 - val_loss: 1.0965
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 12s 458ms/step - accuracy: 0.5331 - loss: 0.9040 - val_accuracy: 0.4776 - val_loss: 1.2661
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 13s 486ms/step - accuracy: 0.5534 - loss: 0.8994 - val_accuracy: 0.4279 - val_loss: 1.0786
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 578ms/step - accuracy: 0.6707 - loss: 0.6943 - val_accuracy: 0.5473 - val_loss: 1.0697
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 615ms/step - accuracy: 0.7431 - loss: 0.6096 - val_accu

메인 코드

In [ ]:
from IPython.display import display
from ipywidgets import FileUpload
from PIL import Image
import io

print("안녕하세요. 얼굴 질병 분석기에 오신 것을 환영합니다.")
print("사진을 입력해주세요")

upload_widget = FileUpload(accept='image/*', multiple=False)

def handle_upload(change):
    if len(change['new']) > 0:
        file_info = change['new'][0]
        file_content = file_info['content']
        
        image = Image.open(io.BytesIO(file_content))
        
        display(image)

# 업로드 위젯에 핸들러 연결
upload_widget.observe(handle_upload, names='value')

# 업로드 위젯 표시
display(upload_widget)

안녕하세요. 얼굴 질병 분석기에 오신 것을 환영합니다.
사진을 입력해주세요


FileUpload(value=(), accept='image/*', description='Upload')